In [1]:
%matplotlib inline

In [2]:
import serial, threading, sys, time, datetime
sys.path.append('/home/pi/Desktop/ADR_NX/Utils')
sys.path.append('/home/pi/Desktop/ADR_NX/Mainframe')
from adr_PID import ADR_PID
import adr_utils, SIM900_controller, SIM921, SIM922
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from HCLib import HeaterController

In [3]:
db = False
HC = HeaterController.HeaterController()
HC.debug = db
HC.establish_comms()

In [4]:
mfc = SIM900_controller.SIM900()
mfc.debug = db
mfc.init_and_open_connection()
mfc.start_input_loop()
mfc.start_output_loop()
s921 = SIM921.SIM921(mfc); s921.debug = db
s922 = SIM922.SIM922(mfc); s922.debug = db
mfc.set_RPER([s922.MODNUM, s921.MODNUM])

Input loop started
Output loop started


In [5]:
mfc.debug = db
HC.set_dutycycle(0)
HC.set_onoff(0)
mfc.get_serial_object().write("SNDT 6,\"SOUT\n\"\n")
mfc.get_serial_object().write("SNDT 1,\"SOUT\n\"\n")

15

============== RERUN THESE 2 AT EVERY TEMP =============

In [22]:
from IPython.display import clear_output
P = 20.0
I = 0.2
D = 0.1
pid = ADR_PID(P,I,D,0)
pid.SetPoint = 24.0
pid.setSlewLimit(0.8)
start = timer()
temps, times, temps_st = ([] for i in range(3))

In [23]:
global runPID
runPID = True
print pid.SetPoint

def PID_thread():
    HC.set_onoff(1)
    HC.set_dutycycle(0)
    global runPID    
    i = 0
    time = 0.0
    while (runPID == True):   
        s922.s922dataevent.wait(5)
        s922.s922dataevent.clear()

        temp = s922.lasttemperature
        temp_st = s921.lasttemperature
        temps_st.append(temp_st)

        if (temp > 40.0):
            continue
            #raise IOError("WRONG TEMP?")
        else:            
            if (time == s922.lastdatatime):
                print("NOT GOOD")
                #HC.set_onoff(0)
                #s922.stop_data_stream()
                #raise IOError("STUCK")   
            time = s922.lastdatatime
            temps.append(temp)
            times.append(timer()-start)    
            pid.update(temp)
            output = pid.output
            if output < 0.0 or output > 100.0:
                continue
            else:
                HC.set_dutycycle(output)                
        print("{:6.4f}|{:6.4f}|{:6.4f}|{}".format(output,temp,temp_st,str(i)))
        i += 1
        if i%15 == 0:
            clear_output()
        
    print("EXITING")

s922.start_data_stream()
s921.start_data_stream()
thread = threading.Thread(name='pidbb',target=PID_thread, args=())
thread.start()

35.8933|23.4400|0.1805|105
35.5987|23.4590|0.1805|106


============== BELOW IS SHUTDOWN COMMAND =============

In [24]:
global runPID
runPID = False
s922.stop_data_stream()
HC.set_onoff(0)
s921.stop_data_stream()
mfc.get_serial_object().write("SNDT 6,\"SOUT\n\"\n")

35.3014|23.4780|0.1805|107


15

NOT GOOD
35.8281|23.4780|0.1811|108
EXITING


In [ ]:
============== EXTRA STUFF =============

In [ ]:
mfc.get_serial_object().write("SNDT 6,\"CURV 0,1\n\"\n")

In [ ]:
t1 = [x if x> 8 else 20.0 for x in temps]
t2 = [x if x< 0.2 else 0.170 for x in temps_st]
t2 = [x if x> 0.112 else 0.170 for x in t2]

In [ ]:
fig = plt.figure()
plt.plot(times,t1,'-')
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.show()

fig = plt.figure()
plt.plot(times,t2,'-')
plt.xlabel("Time (s)")
plt.ylabel("Temperature (K)")
plt.show()

In [ ]:
mfc.stop_input_loop()
mfc.stop_output_loop()